In [6]:
import logging
import os

import numpy as np

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
logging.getLogger().setLevel(logging.ERROR)
import tensorflow as tf
tf.get_logger().setLevel(logging.ERROR)

import sys
sys.path.append("../src")

from toolz import keyfilter

from Evaluation import create_confusion_matrix_plot_from_csv
from LoadData import CurrentDatasets
from EnsembleBuilder import EnsembleBuilder

In [7]:
dataset_names = [dataset.value for dataset in CurrentDatasets]

# in a nested list all elements on the same level get equal weights
# eg [foo, [bar, bar, bar, [baz, baz]]] is weighted [1/2, [1/8, 1/8, 1/8, [1/16, 1/16]]]

ensembles = {"All": [[f"MLP-{number}" for number in range(0,10)],
                     [f"FCN-{number}" for number in range(0,10)],
                     [f"MCDCNN_improved-{number}" for number in range(0,10)],
                     [f"Encoder-{number}" for number in range(0,10)],
                     [f"Resnet-{number}" for number in range(0,10)]],
             "MLP10": [f"MLP-{number}" for number in range(0,10)],
             "FCN10": [f"FCN-{number}" for number in range(0,10)],
             "MCDCNN_improved10": [f"MCDCNN_improved-{number}" for number in range(0,10)],
             "Encoder10": [f"Encoder-{number}" for number in range(0,10)],
             "Resnet10": [f"Resnet-{number}" for number in range(0,10)],
             "NNE": [[f"Resnet-{number}" for number in range(0,10)],
                     [f"FCN-{number}" for number in range(0,10)],
                     [f"Encoder-{number}" for number in range(0,10)]],
             "Best4": [[f"Resnet-{number}" for number in range(0,10)],
                       [f"FCN-{number}" for number in range(0,10)],
                       [f"MLP-{number}" for number in range(0,10)],
                       [f"Encoder-{number}" for number in range(0,10)]],
             "FCNpretrain": []}
csv_name = '../results/ensembles_with_augmentation.csv'
dataset_names = list(filter(lambda n: n not in [], dataset_names)) # remove electric devices
dataset_names

['cricket_y',
 'distal_phalanax_tw',
 'egg_five_days',
 'electric_devices',
 'face_ucr',
 'mote_strain',
 'power_cons',
 'sony_robot',
 'swedish_leaf',
 'synthetic_control']

In [8]:
used_ensembles=ensembles
#used_ensembles=keyfilter(lambda ensemble: ensemble in ["All", "MCDCNN_improved10"], ensembles)
used_ensembles

{'All': [['MLP-0',
   'MLP-1',
   'MLP-2',
   'MLP-3',
   'MLP-4',
   'MLP-5',
   'MLP-6',
   'MLP-7',
   'MLP-8',
   'MLP-9'],
  ['FCN-0',
   'FCN-1',
   'FCN-2',
   'FCN-3',
   'FCN-4',
   'FCN-5',
   'FCN-6',
   'FCN-7',
   'FCN-8',
   'FCN-9'],
  ['MCDCNN_improved-0',
   'MCDCNN_improved-1',
   'MCDCNN_improved-2',
   'MCDCNN_improved-3',
   'MCDCNN_improved-4',
   'MCDCNN_improved-5',
   'MCDCNN_improved-6',
   'MCDCNN_improved-7',
   'MCDCNN_improved-8',
   'MCDCNN_improved-9'],
  ['Encoder-0',
   'Encoder-1',
   'Encoder-2',
   'Encoder-3',
   'Encoder-4',
   'Encoder-5',
   'Encoder-6',
   'Encoder-7',
   'Encoder-8',
   'Encoder-9'],
  ['Resnet-0',
   'Resnet-1',
   'Resnet-2',
   'Resnet-3',
   'Resnet-4',
   'Resnet-5',
   'Resnet-6',
   'Resnet-7',
   'Resnet-8',
   'Resnet-9']],
 'MLP10': ['MLP-0',
  'MLP-1',
  'MLP-2',
  'MLP-3',
  'MLP-4',
  'MLP-5',
  'MLP-6',
  'MLP-7',
  'MLP-8',
  'MLP-9'],
 'FCN10': ['FCN-0',
  'FCN-1',
  'FCN-2',
  'FCN-3',
  'FCN-4',
  'FCN-5',
  

In [9]:
ens_builder = EnsembleBuilder(dataset_names=dataset_names, 
                              ensembles=used_ensembles, 
                              verbose=True,
                              models_path="../models/without_augmentation/",
                              datasets_path="../datasets/")
ens_builder.run_ensembles(augmentation=False).to_csv(csv_name)

1/10:	cricket_y
	All


/home/felix/miniconda3/envs/tf/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


	MLP10
	FCN10
	MCDCNN_improved10
	Encoder10


/home/felix/miniconda3/envs/tf/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


	Resnet10
	NNE


...

/home/felix/miniconda3/envs/tf/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


	Resnet10
	NNE


/home/felix/miniconda3/envs/tf/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


	Best4


/home/felix/miniconda3/envs/tf/lib/python3.10/site-packages/keras/layers/core/lambda_layer.py:324: UserWarning: ModelBuilder is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(


In [ ]:
create_confusion_matrix_plot_from_csv(csv_name, verbose=True)

In [ ]:
create_confusion_matrix_plot_from_csv("../results/train__Encoder_Encoder_Encoder_Encoder_Encoder_Encoder_Encoder_Encoder_Encoder_Encoder_11_datasets.csv", verbose=True)